In [ ]:
%%html
<style>
.output_wrapper button.btn.btn-default,
.output_wrapper .ui-dialog-titlebar {
  display: none;
}
</style>

<div>
    <img src="figures/rwth_ient_logo@2x.png" style="float: right;height: 5em;">
</div>

# Interactive Demo

This notebook shows an easy interactive demo.

First, import necessary packets. 

* `ipywidgets` has a great documentation [https://ipywidgets.readthedocs.io/en/stable/user_guide.html] with a lot of examples [https://github.com/jupyter-widgets/ipywidgets/tree/master/docs/source/examples].
* `ient_plots` creates mainly axis labels styles which are commonly used in IENT.

In [ ]:
%matplotlib notebook

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from IPython.display import clear_output, display, HTML
from IPython.display import Markdown as md

import numpy as np
from scipy import integrate
from scipy import signal

from ient_plots import *

Main plot routine. First, random values are sampled, then binned by a histogram and finally plotted.

In [ ]:
(t,deltat) = np.linspace(-4, 4, 1001, retstep=True) # t-axis
lim_hist = 7 # limit of histogram (the total range would be from -lim_hist to lim_hist)


def update_plots(axs,ms,sigmas,dist_type):
    
    # Sample from given distribution
    if dist_type == 'Normal':
        s = np.random.normal(ms,sigmas,len(t))
    elif dist_type == 'Uniform':
        c = np.sqrt(12*sigmas**2)
        a = -0.5*c+ms
        b = 0.5*c+ms
        s = np.random.uniform(a,b,len(t))
    
    # Calculate histogram to estimate p_s(x)
    ps,bins = np.histogram(s,bins='auto',range=(-lim_hist,lim_hist),density=True)
    x = (bins[:-1] + bins[1:]) / 2 # x-axis
    
    if axs[0].lines: # check if lines exist
        # If yes, replace only xdata and ydata since plt.plot() takes longer time
        # Usually, these functions should not take too much processing time
        axs[0].lines[0].set_xdata(t); 
        axs[0].lines[0].set_ydata(s);

        axs[1].lines[0].set_xdata(x); 
        axs[1].lines[0].set_ydata(ps);
        
    else:
        # If no, call plot() and decorate axes
        # Usually, these functions take some processing time
        
        # First axis displays stoch. process s(t) in time-domain
        ax = axs[0]
        ax.plot(t,s)
        ax.set_xlabel(r'$\rightarrow t$'); ax.set_ylabel(r'$\uparrow s(t)$'); 
        ax.set_yticks(list(range(-lim_hist,lim_hist,2)))
        ax.set_xlim([-3.5,5.5]); ax.set_ylim([-5.5,6.5])
        ient_axis(ax)

        # Second axis displays prob. density function p_s(x)
        ax = axs[1]
        ax.plot(x,ps)
        ax.set_xlabel(r'$\rightarrow x$'); ax.set_ylabel(r'$\uparrow p_s(x)$'); 
        ax.set_xticks(list(range(-lim_hist,lim_hist,2)))
        ax.set_xlim(axs[0].get_ylim()); ax.set_ylim([-0.05,0.75])
        ient_axis(ax)
        #axs[1].set_xticks(axs[0].get_yticks())

Next, the widgets are created at once with `interactive`, which on update calls `update_plots` defined in the cell above.  

In [ ]:
# Open figure
fig, axs = plt.subplots(1, 2, figsize=(8,3))

# Interactive takes a function as first input and maps the inputs of the function as widgets.
wdgts = interactive(update_plots,
                axs = fixed(axs), # no widget for axs :)
                ms = widgets.FloatSlider(min=-2, max=2, step=0.1, value=0, description='$m_s$:'),
                sigmas = widgets.FloatSlider(min=0.1, max=2, step=0.1, value=1, description='$\sigma_s$:'),
                dist_type = widgets.Dropdown(options=['Normal', 'Uniform'], description='Distribution:'));

# Additionally, we create an update button which manually updates update_plots
update_button = widgets.Button(description="Update")
def on_button_clicked(b):
    wdgts.update()
update_button.on_click(on_button_clicked)

# Group widgets with a horizontal box
hbox = widgets.HBox(children=[wdgts, update_button])
display(hbox)

Here, we could write some descriptive text to explain which signals are plotted and how they behave.